[Home](../index.ipynb) / ADC-Wandler
***
<span style="font-size:20pt;">ADC-Wandler</span>

# ADC-Wandler
## Grundlagen
Die Controller besitzten einen Analog-Digital-Wandler (ADC) der Spannungen in Zahlen umwandelt:  
Beim ESP32 entpricht $0 V$ dem Wert $0$ und $3,3 V$ dem Wert $65535 = 2^{16} - 1$

<span style="color:red">**Achtung:**</span>
* ESP32, also HelTec und NodeMCU: es darf <span style="color:red">**nie** eine Spannung **> 3,3 V**</span> an einen Pin angelegt werden!  
  Besonders <span style="color:red">**gefährlich:**</span> der 5 V Pin und der 3,3 V Pin liegen direkt nebeneinander!
* Esp8266, also Croduino: am **Pin A** darf <span style="color:red">**nie** eine Spannung **> 1,0 V**</span> angelegt werden, am **Pin A_e** <span style="color:red">**nie** eine Spannung **> 5,0 V**</span>!

### Spannungsverläufe messen
(HelTec, NodeMCU mit `ADC.ATTN_11DB`)

[<img src="images/ADC_Principle.png" width="400">](images/ADC_Principle.png)

Genaueres zur Funktionsweise von ADCs siehe z.B. [Wikipedia](https://de.wikipedia.org/wiki/Analog-Digital-Umsetzer).

### HeltTec, NodeMCU
**Pins mit ADC:**  
Beim ESP32 ist ADC an den Pins 32-39 (ADC Block 1) and den Pins 0, 2, 4, 12-15 und 25-27 (ADC Block 2) verfügbar.  
Details siehe [micropython.org: ADC](https://docs.micropython.org/en/latest/esp32/quickref.html#adc-analog-to-digital-conversion).   

**Messwerte:**  
$0$ entspricht $0 V$ bezüglich `GND` und $2^{16} -1 \,=\, 65535$ entspricht der **internen Referenzspannung** von $1,1 \pm 0,1V$ bei $0\, dB$ Dämpfung.  
Für andere Messbereiche kann die **Dämpfung** (engl. Attenuation) gesetzt werden, so dass sich der Messbereich bis auf $3,3 V$ erweitern lässt.
<!--
* $0 \; dB \Rightarrow \; 1,1 V\; \mathrel{\hat{=}} \;65535$
* $2.5 \; dB \Rightarrow \; 1,4669 V\; \mathrel{\hat{=}} \;65535$
* $6 \; dB \Rightarrow \; 2,1979 V\; \mathrel{\hat{=}} \;65535$
* $11 \; dB \Rightarrow \; 3,9029 V\; \mathrel{\hat{=}} \;65535$

Formeln für die Dämpfung $a$ in $dB$ (dezi Bel):
* Spannung: $a \, =\, 20 \cdot log_{10} \left(\frac{U_{in}}{U_{out}}\right)$  
* Strom: $a \, =\, 20 \cdot log_{10} \left(\frac{I_{in}}{I_{out}}\right)$  
* Leistung: $a \, =\, 10 \cdot log_{10} \left(\frac{P_{in}}{P_{out}}\right)$  
-->

**Typischer Code:**  
```python
adc = ADC(Pin(32)) # HelTec, NodeMCU
# adc.atten(ADC.ATTN_0DB)   # [1,1 V ~ 65535] (default Wert) (eigene Messungen)
# adc.atten(ADC.ATTN_2_5DB) # [1,5 V ~ 65535]
# adc.atten(ADC.ATTN_6DB)   # [2,3 V ~ 65535] (eigene Messungen)
adc.atten(ADC.ATTN_11DB)    # [3,3 V ~ 65535] (eigene Messungen)
iValue = adc.read_u16()
```

Achtung: Laut [micropython.org: ADC](https://docs.micropython.org/en/latest/esp32/quickref.html#adc-analog-to-digital-conversion) sollte auch
`adc = ADC(Pin(32), atten=ADC.ATTN_11DB)` klappen, aber das hat (2022-02-20) keinen Effekt).

### Esp8266 Croduino
**Pins mit ADC:**  
Beim Esp8266 Croduino sind es die Pins A (1,0V) und A_e (5,0V) mit 7 Bit Auflösung.  
Details siehe [micropython.org: ADC](https://docs.micropython.org/en/latest/esp8266/quickref.html#adc-analog-to-digital-conversion).




## Beispielprogramm

In [ ]:
%serialconnect

from machine import ADC, Pin
import display
display = display.Display()

from time import sleep

#adc = ADC(0)       # Croduino
adc = ADC(Pin(32)) # HelTec, NodeMCU
adc.atten(ADC.ATTN_11DB)   # [3,3 V ~ 65535]


while True:
    display.clear()
    
    iValue = adc.read_u16()  # read a raw analog value in the range 0-65535 (= 2^16 - 1)
    #iValue = adc.read()      # Croduino: read a raw analog value in the range 0-511 (= 2^7 - 1)
    
    display.text( str(iValue), 0, 0 )
    display.show()
    

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.
.

Wenn Sie nun den Pin 32 auf $0 V$ oder $3,3 V$ setzten, wird entweder $0$ oder $65535$ angezeigt.

# Dimmerschaltung mit Photowiderstand

## Motivation
[<img src="images/NachtflugverbotTornadosZeit.png" width="480">](images/NachtflugverbotTornadosZeit.png)

## Aufgabe
Mit einem Arduino kompatiblen Board soll eine automatische Dimmerschaltung entwickelt werden die obiges Problem behebt:  
die Beleuchtung soll bei weniger Umgebungslicht automatisch schwächer werden.

### Photowiderstand
Die Messung der Umgebungshelligkeit kann z.B. mit einem Fotowiderstand (**LDR** für **L**ight **D**ependent **R**esistor) <img src="images/PhotoResistor.png" width="50"> erfolgen:  
der vom Set weist bei Dunkelheit einen Widerstand von etwa $10 kΩ$ auf und bei Tageslicht einen Widerstand von etwa $1 kΩ$ (je nach Typ).

### Spannungsteiler
Widerstände kann der Arduino nicht messen, also wird ein Spannungsteiler verwendet, damit aus dem Strom durch den LDR eine messbare Spannung wird:

[<img src="images/Spannungsteiler_R.png" width="160">](images/Spannungsteiler_R.png)

Im Spannungsteiler mit $U\,=\,U_1 + U_2$ gilt: durch $R_1$ und $R_2$ fließt der Strom $I$ und es gilt: $\frac{R_1}{R_2}\,=\,\frac{U_1}{U_2}$.

Mit dem LDR ergibt sich dann folgende Schaltung:

[<img src="images/Spannungsteiler_LDR.png" width="200">](images/Spannungsteiler_LDR.png)

Hier entspricht der Photowiderstand $R_2$ und der Widerstand entspricht $R_1$.

### Arbeitsauftrag
Die Beleuchtung soll durch eine externe blaue LED realisiert werden, der Umgebungshelligkeitssensor mit einem LDR.

Die minimale und die maximale Helligkeit sollen jeweils über eine Variable festgelegt werden (sie sollen später mit einem Rotary-Encoder eingestellt werden können).